<a href="https://colab.research.google.com/github/SNOOthiking/SHOPPING/blob/snoo/KoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 가상환경 삭제 : conda remove -n 가상환경이름 --all
TPU 환경으로 돌려야 빠름 
---



In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 101 kB 9.4 MB/s 
     |████████████████████████████████| 6.6 MB 40.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers
transformers.__version__

'4.20.1'

In [4]:
import pandas as pd 
import numpy as np
import torch # pytorch install 해야함
import io
import os
from tqdm import tqdm
import tensorflow as tf
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [5]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.48.168.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.48.168.210:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [6]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/shopping/dataset/train.csv')  

In [9]:
train

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5
...,...,...,...
24995,24995,일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여,5
24996,24996,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.,2
24997,24997,향이 좋아요!! 다른 향도 시켜보고 싶어요,4
24998,24998,펼쳐보니 끝부분쯤에 50cm정도 찟어져 있습니다 날짜관계로 구냥 사용합니다,2


In [10]:
train_data, val_data = train_test_split(train, test_size=0.25, 
                                        stratify = train.target,
                                        random_state =0) # 25프로로 설정
# 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
train_data = train_data.reset_index().drop('index', axis=1)
val_data = val_data.reset_index().drop('index', axis=1)


In [12]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')


Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
tokenizer.encode('안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다')

[25906,
 8702,
 7801,
 25856,
 9265,
 7162,
 16071,
 15460,
 11510,
 10973,
 7898,
 8135,
 9212,
 7791,
 8092,
 9241,
 7172,
 7182]

In [14]:
tokenizer.tokenize('안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다')

['▁안녕',
 '하',
 '세',
 '요.',
 '▁저',
 '는',
 '▁데이터',
 '분석',
 '가가',
 '▁되고',
 '싶',
 '은',
 '▁최',
 '선',
 '우',
 '▁입',
 '니',
 '다']

In [15]:
tokenizer.decode(tokenizer.encode('안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다'))

'안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다'

In [16]:
max_seq_len = 100 # 제일 긴 문장의 길이

In [ ]:
tokenizer.encode('안녕하세요. 저는 데이터분석가가 되고싶은 최선우 입니다',max_length=max_seq_len,pad_to_max_length = True)

In [18]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):
    max_seq_len =100
    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [19]:
def make_sequences(texts):
    seq_list = []
    for text in texts:
        tokens = [tokenizer.bos_token]+ tokenizer.tokenize(text) + [tokenizer.eos_token]
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        pad = pad_sequences([input_id],maxlen=max_seq_len,value=tokenizer.pad_token_id,padding='post')[0]
        seq_list.append(pad)
    seq_list = np.array(seq_list,dtype=int)
    return seq_list

In [94]:

class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.classifier = tf.keras.layers.Dense(8, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='relu',
                                                name='classifier')
        self.classifier = tf.keras.layers.Dense(16, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='relu',
                                                name='classifier')
        self.classifier = tf.keras.layers.Dense(4,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        cls_token = self.dropout(cls_token)
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
model.

In [21]:
train_data
val_data

,id,reviews,target
0,10749,심플하면서 깔끔하고 가격 저렴하고 배송 빠르고~ 최고네요 ^^,5
1,12258,재구매 아주 좋아요 손톱관리하기가 너무 좋아요~~,5
2,16351,조립도 쉽고 디자인 너무 예뻐요ㅎㅎ,5
3,9402,재구매 배송 빠르고 불량품 없어서 좋아요,5
4,7722,배송은 늦었지만 수령후 설치해서사용중인데 모니터 고정부분이 움직일때 좌우로 움직이네...,4
...,...,...,...
6245,24390,싸서샀는데별로임 질떨어짐,2
6246,16848,잘 받아 보았습니다,2
6247,20355,배송이 너무 늦게오네요..,2
6248,15497,향이...별로예요 ㅠㅠㅠ,1


In [22]:
train_X, train_y = convert_examples_to_features(train_data['reviews'], train_data['target'], max_seq_len=max_seq_len, tokenizer=tokenizer)


100%|██████████| 18750/18750 [00:03<00:00, 4978.18it/s]


In [23]:
val_X, val_y = convert_examples_to_features(val_data['reviews'], val_data['target'], max_seq_len=max_seq_len, tokenizer=tokenizer)


100%|██████████| 6250/6250 [00:01<00:00, 4962.35it/s]


In [24]:
tokenizer.decode(train_X[0])

'</s> 상품이 저가 원하는대로 왔어요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM 

In [27]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)


Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.0.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.2.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [28]:
layers = tf.keras.layers.Input(shape=(max_seq_len),dtype=tf.int32)
outputs = model([layers])

In [95]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.BinaryCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'lm_head.weight', 'transformer.h.0.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.2.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [34]:
train_y =pd.get_dummies(train_y)

In [35]:
train_X

array([[    1, 41276,  9265, ...,     3,     3,     3],
       [    1,  9774,  7623, ...,     3,     3,     3],
       [    1, 12371, 12659, ...,     3,     3,     3],
       ...,
       [    1,  9306,  6853, ...,     3,     3,     3],
       [    1, 12224,  7770, ...,     3,     3,     3],
       [    1, 10558,  8382, ...,     3,     3,     3]])

In [75]:
model.fit(train_X,train_y)

586/586 [==============================] - 117s 76ms/step - loss: 0.4355 - accuracy: 0.6000


In [37]:
val_y = pd.get_dummies(val_y)

In [76]:
result = model.evaluate(val_X,val_y)

196/196 [==============================] - 20s 41ms/step - loss: 0.3469 - accuracy: 0.6768


In [77]:
result

[0.3468938171863556, 0.676800012588501]

In [40]:

def sentiment_predict(new_sentence):

  bos_token = [tokenizer.bos_token]
  eos_token = [tokenizer.eos_token]
  tokens = bos_token + tokenizer.tokenize(new_sentence) + eos_token
  input_id = tokenizer.convert_tokens_to_ids(tokens)
  input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]
  input_id = np.array([input_id])
  score = model.predict(input_id)[0][0]

  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [41]:
sentiment_predict(train['reviews'][0])


99.64% 확률로 부정 리뷰입니다.



In [78]:
train_Xall, train_yall =  convert_examples_to_features(train['reviews'], train['target'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 25000/25000 [00:05<00:00, 4609.30it/s]


In [79]:
train_yall = pd.get_dummies(train_yall)

In [96]:
model.fit(train_Xall,train_yall)

782/782 [==============================] - 172s 128ms/step - loss: 0.4356 - accuracy: 0.6134


In [46]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/shopping/dataset/test.csv')

In [47]:
test

,id,reviews
0,0,채소가 약간 시들어 있어요
1,1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2,2,부들부들 좋네요 입어보고 시원하면 또 살게요
3,3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4,4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요
...,...,...
24995,24995,사용해보니 좋아요~^^
24996,24996,저렴한가격에. 질좋고. 핏좋고. 너무. 이쁘게. 입고다녀요..
24997,24997,세트상품이라고 써있어서 그런줄 알고 구매했더니 단품이었네요 낚인 느낌도 들고 그러네...
24998,24998,역시 로네펠트!! 좋아요.


In [50]:
test_X = make_sequences(test['reviews'])

In [55]:
len(test_X[0])

100

In [97]:
test_pred = model.predict(test_X).argmax(axis=1)

In [82]:
test_pred

array([1, 0, 3, ..., 1, 3, 1])

In [102]:
test_result = pd.Series(test_pred).replace(3,5).replace(2,4).replace(1,2).replace(0,1)

In [106]:
(test_result==1).sum()

1576

In [107]:
submission2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/shopping/dataset/sample_submission.csv')

In [108]:
submission2['target'] =test_result

In [109]:
submission2.to_csv('/content/drive/MyDrive/Colab Notebooks/shopping/dataset/submission6.csv',index=False)